In [1]:
import findspark
findspark.init('/usr/hdp/current/spark2-client')
findspark.find()

'/usr/hdp/current/spark2-client'

In [2]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("yarn").appName("ch07Aggreagations").getOrCreate()

In [3]:
sc = spark.sparkContext

In [4]:
for x in sc._conf.getAll():
    if '/proxy/' in x[1]:
        print(x[1])

In [5]:
df = spark.read.format("csv")\
  .option("header", "true")\
  .option("inferSchema", "true")\
  .load("/user/kranthidr/dataSets/spark-guide/retail-data/all/*.csv")\
  .coalesce(5)
df.cache()
df.createOrReplaceTempView("dfTable")

In [6]:
df.printSchema()

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: string (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: integer (nullable = true)
 |-- Country: string (nullable = true)



In [7]:
# COMMAND ----------

from pyspark.sql.functions import count
df.select(count("StockCode")).show() # 541909

+----------------+
|count(StockCode)|
+----------------+
|          541909|
+----------------+



In [8]:
# COMMAND ----------

from pyspark.sql.functions import countDistinct
df.select(countDistinct("StockCode")).show() # 4070

+-------------------------+
|count(DISTINCT StockCode)|
+-------------------------+
|                     4070|
+-------------------------+



In [9]:
# COMMAND ----------

from pyspark.sql.functions import approx_count_distinct
df.select(approx_count_distinct("StockCode", 0.1)).show() # 3364

+--------------------------------+
|approx_count_distinct(StockCode)|
+--------------------------------+
|                            3364|
+--------------------------------+



In [10]:
# COMMAND ----------

from pyspark.sql.functions import first, last
df.select(first("StockCode"), last("StockCode")).show()

+-----------------------+----------------------+
|first(StockCode, false)|last(StockCode, false)|
+-----------------------+----------------------+
|                 85123A|                 22138|
+-----------------------+----------------------+



In [11]:
# COMMAND ----------

from pyspark.sql.functions import min, max
df.select(min("Quantity"), max("Quantity")).show()

+-------------+-------------+
|min(Quantity)|max(Quantity)|
+-------------+-------------+
|       -80995|        80995|
+-------------+-------------+



In [12]:
# COMMAND ----------

from pyspark.sql.functions import sum
df.select(sum("Quantity")).show() # 5176450

+-------------+
|sum(Quantity)|
+-------------+
|      5176450|
+-------------+



In [13]:
# COMMAND ----------

from pyspark.sql.functions import sumDistinct
df.select(sumDistinct("Quantity")).show() # 29310

+----------------------+
|sum(DISTINCT Quantity)|
+----------------------+
|                 29310|
+----------------------+



In [14]:
df.select("Quantity").groupBy("Quantity").count().show()

+--------+-----+
|Quantity|count|
+--------+-----+
|     496|    2|
|    -125|    3|
|     148|    1|
|     -35|   14|
|    -975|    1|
|     392|    2|
|     540|    2|
|    -967|    1|
|      31|   54|
|     -78|    2|
|    3000|    2|
|    -590|    1|
|      85|    7|
|     137|    1|
|      65|   14|
|    -192|    7|
|    4000|    1|
|      53|   19|
|     588|    1|
|    -164|    2|
+--------+-----+
only showing top 20 rows



In [15]:
df.select("Quantity").groupBy("Quantity").count().select(sum("count")).show() # 29310

+----------+
|sum(count)|
+----------+
|    541909|
+----------+



In [16]:
df.select("Quantity").count()

541909

In [17]:
# COMMAND ----------

from pyspark.sql.functions import sum, count, avg, expr

df.select(
    count("Quantity").alias("total_transactions"),
    sum("Quantity").alias("total_purchases"),
    avg("Quantity").alias("avg_purchases"),
    expr("mean(Quantity)").alias("mean_purchases"))\
  .selectExpr(
    "total_purchases/total_transactions",
    "avg_purchases",
    "mean_purchases").show()

+--------------------------------------+----------------+----------------+
|(total_purchases / total_transactions)|   avg_purchases|  mean_purchases|
+--------------------------------------+----------------+----------------+
|                      9.55224954743324|9.55224954743324|9.55224954743324|
+--------------------------------------+----------------+----------------+



In [18]:
# COMMAND ----------

from pyspark.sql.functions import var_pop, stddev_pop
from pyspark.sql.functions import var_samp, stddev_samp
df.select(var_pop("Quantity"), var_samp("Quantity"),
  stddev_pop("Quantity"), stddev_samp("Quantity")).show()

+------------------+------------------+--------------------+---------------------+
| var_pop(Quantity)|var_samp(Quantity)|stddev_pop(Quantity)|stddev_samp(Quantity)|
+------------------+------------------+--------------------+---------------------+
|47559.303646609354| 47559.39140929905|  218.08095663447864|   218.08115785023486|
+------------------+------------------+--------------------+---------------------+



In [19]:
# COMMAND ----------

from pyspark.sql.functions import skewness, kurtosis
df.select(skewness("Quantity"), kurtosis("Quantity")).show()

+--------------------+------------------+
|  skewness(Quantity)|kurtosis(Quantity)|
+--------------------+------------------+
|-0.26407557610527843|119768.05495536518|
+--------------------+------------------+



In [20]:
# COMMAND ----------

from pyspark.sql.functions import corr, covar_pop, covar_samp
df.select(corr("InvoiceNo", "Quantity"), covar_samp("InvoiceNo", "Quantity"),
    covar_pop("InvoiceNo", "Quantity")).show()

+-------------------------+-------------------------------+------------------------------+
|corr(InvoiceNo, Quantity)|covar_samp(InvoiceNo, Quantity)|covar_pop(InvoiceNo, Quantity)|
+-------------------------+-------------------------------+------------------------------+
|     4.912186085637639E-4|             1052.7280543913773|            1052.7260778752732|
+-------------------------+-------------------------------+------------------------------+



In [21]:
# COMMAND ----------

from pyspark.sql.functions import collect_set, collect_list
df.agg(collect_set("Country"), collect_list("Country")).show()

+--------------------+---------------------+
|collect_set(Country)|collect_list(Country)|
+--------------------+---------------------+
|[Portugal, Italy,...| [United Kingdom, ...|
+--------------------+---------------------+



In [22]:
# df.agg(collect_set("Country"), collect_list("Country")).show(5, False)
# IOPub data rate exceeded.
# The notebook server will temporarily stop sending output
# to the client in order to avoid crashing it.
# To change this limit, set the config variable
# `--NotebookApp.iopub_data_rate_limit`.

In [23]:
# COMMAND ----------

from pyspark.sql.functions import count

df.groupBy("InvoiceNo").agg(
    count("Quantity").alias("quan"),
    expr("count(Quantity)")).show(5)

+---------+----+---------------+
|InvoiceNo|quan|count(Quantity)|
+---------+----+---------------+
|   563020|  24|             24|
|   565747|  10|             10|
|   566248|   8|              8|
|   566431|  18|             18|
|   567163|  14|             14|
+---------+----+---------------+
only showing top 5 rows



In [24]:
# COMMAND ----------

df.groupBy("InvoiceNo").agg(expr("avg(Quantity)"),expr("stddev_pop(Quantity)"))\
  .show(5)

+---------+------------------+--------------------+
|InvoiceNo|     avg(Quantity)|stddev_pop(Quantity)|
+---------+------------------+--------------------+
|   536596|               1.5|  1.1180339887498947|
|   536938|33.142857142857146|  20.698023172885524|
|   537252|              31.0|                 0.0|
|   537691|              8.15|   5.597097462078001|
|   538041|              30.0|                 0.0|
+---------+------------------+--------------------+
only showing top 5 rows



In [38]:
df.select(col("InvoiceDate")).show(5, False)

+--------------+
|InvoiceDate   |
+--------------+
|12/1/2010 8:26|
|12/1/2010 8:26|
|12/1/2010 8:26|
|12/1/2010 8:26|
|12/1/2010 8:26|
+--------------+
only showing top 5 rows



In [39]:
# COMMAND ----------
from pyspark.sql.functions import col, to_date

dfWithDate = df.withColumn("date", to_date(col("InvoiceDate"), "MM/d/yyyy H:mm"))
dfWithDate.createOrReplaceTempView("dfWithDate")

In [40]:
cols = dfWithDate.columns

In [41]:
cols.remove("Description")

In [42]:
cols

['InvoiceNo',
 'StockCode',
 'Quantity',
 'InvoiceDate',
 'UnitPrice',
 'CustomerID',
 'Country',
 'date']

In [43]:
cols_ = map(lambda x:col(x), cols)

In [44]:
cols_

[Column<InvoiceNo>,
 Column<StockCode>,
 Column<Quantity>,
 Column<InvoiceDate>,
 Column<UnitPrice>,
 Column<CustomerID>,
 Column<Country>,
 Column<date>]

In [45]:
cols_.append(expr("*"))

In [46]:
cols_

[Column<InvoiceNo>,
 Column<StockCode>,
 Column<Quantity>,
 Column<InvoiceDate>,
 Column<UnitPrice>,
 Column<CustomerID>,
 Column<Country>,
 Column<date>,
 Column<unresolvedstar()>]

In [56]:
dfWithDate.select(*cols_).columns

['InvoiceNo',
 'StockCode',
 'Quantity',
 'InvoiceDate',
 'UnitPrice',
 'CustomerID',
 'Country',
 'date',
 'InvoiceNo',
 'StockCode',
 'Description',
 'Quantity',
 'InvoiceDate',
 'UnitPrice',
 'CustomerID',
 'Country',
 'date']

In [57]:
dfWithDate.select(cols_).columns

['InvoiceNo',
 'StockCode',
 'Quantity',
 'InvoiceDate',
 'UnitPrice',
 'CustomerID',
 'Country',
 'date',
 'InvoiceNo',
 'StockCode',
 'Description',
 'Quantity',
 'InvoiceDate',
 'UnitPrice',
 'CustomerID',
 'Country',
 'date']

In [58]:
dfWithDate.select(cols).show(5,False)

+---------+---------+--------+--------------+---------+----------+--------------+----------+
|InvoiceNo|StockCode|Quantity|InvoiceDate   |UnitPrice|CustomerID|Country       |date      |
+---------+---------+--------+--------------+---------+----------+--------------+----------+
|536365   |85123A   |6       |12/1/2010 8:26|2.55     |17850     |United Kingdom|2010-12-01|
|536365   |71053    |6       |12/1/2010 8:26|3.39     |17850     |United Kingdom|2010-12-01|
|536365   |84406B   |8       |12/1/2010 8:26|2.75     |17850     |United Kingdom|2010-12-01|
|536365   |84029G   |6       |12/1/2010 8:26|3.39     |17850     |United Kingdom|2010-12-01|
|536365   |84029E   |6       |12/1/2010 8:26|3.39     |17850     |United Kingdom|2010-12-01|
+---------+---------+--------+--------------+---------+----------+--------------+----------+
only showing top 5 rows



In [59]:
dfWithDate.printSchema()

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: string (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: integer (nullable = true)
 |-- Country: string (nullable = true)
 |-- date: date (nullable = true)



In [60]:
# COMMAND ----------

from pyspark.sql.window import Window
from pyspark.sql.functions import desc

windowSpec = Window.partitionBy("CustomerId", "date").orderBy(desc("Quantity"))\
  .rowsBetween(Window.unboundedPreceding, Window.currentRow)

In [61]:
type(windowSpec)

pyspark.sql.window.WindowSpec

In [62]:
# Window.
# Window.partitionBy("CustomerId", "date")\
#Window.orderBy(desc("Quantity"))\
#Window.rowsBetween(Window.unboundedPreceding, Window.currentRow)
#Window.unboundedFollowing
#Window.rangeBetween(start,end)

In [63]:
# COMMAND ----------

from pyspark.sql.functions import max
maxPurchaseQuantity = max(col("Quantity")).over(windowSpec)

In [64]:
maxPurchaseQuantity

Column<max(Quantity) OVER (PARTITION BY CustomerId, date ORDER BY Quantity DESC NULLS LAST ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW)>

In [65]:
# COMMAND ----------

from pyspark.sql.functions import dense_rank, rank
purchaseDenseRank = dense_rank().over(windowSpec)
purchaseRank = rank().over(windowSpec)

In [66]:
purchaseDenseRank

Column<DENSE_RANK() OVER (PARTITION BY CustomerId, date ORDER BY Quantity DESC NULLS LAST ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW)>

In [67]:
purchaseRank

Column<RANK() OVER (PARTITION BY CustomerId, date ORDER BY Quantity DESC NULLS LAST ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW)>

In [68]:
# COMMAND ----------

from pyspark.sql.functions import col

windowDF = dfWithDate.where("CustomerId IS NOT NULL").orderBy("CustomerId")\
  .select(
    col("CustomerId"),
    col("date"),
    col("Quantity"),
    purchaseRank.alias("quantityRank"),
    purchaseDenseRank.alias("quantityDenseRank"),
    maxPurchaseQuantity.alias("maxPurchaseQuantity"))

In [69]:
windowDF.show()

+----------+----------+--------+------------+-----------------+-------------------+
|CustomerId|      date|Quantity|quantityRank|quantityDenseRank|maxPurchaseQuantity|
+----------+----------+--------+------------+-----------------+-------------------+
|     12346|2011-01-18|   74215|           1|                1|              74215|
|     12346|2011-01-18|  -74215|           2|                2|              74215|
|     12347|2010-12-07|      36|           1|                1|                 36|
|     12347|2010-12-07|      30|           2|                2|                 36|
|     12347|2010-12-07|      24|           3|                3|                 36|
|     12347|2010-12-07|      12|           4|                4|                 36|
|     12347|2010-12-07|      12|           4|                4|                 36|
|     12347|2010-12-07|      12|           4|                4|                 36|
|     12347|2010-12-07|      12|           4|                4|             

In [70]:
windowDF.groupBy("CustomerId").count().orderBy(col("count").desc()).show()

+----------+-----+
|CustomerId|count|
+----------+-----+
|     17841| 7983|
|     14911| 5903|
|     14096| 5128|
|     12748| 4642|
|     14606| 2782|
|     15311| 2491|
|     14646| 2085|
|     13089| 1857|
|     13263| 1677|
|     14298| 1640|
|     15039| 1508|
|     14156| 1420|
|     18118| 1284|
|     14159| 1212|
|     14796| 1165|
|     15005| 1160|
|     16033| 1152|
|     14056| 1128|
|     14769| 1094|
|     17511| 1076|
+----------+-----+
only showing top 20 rows



In [71]:
windowDF.orderBy(col("CustomerId"), col("date")).where("CustomerId = 14096").show(50)

+----------+----------+--------+------------+-----------------+-------------------+
|CustomerId|      date|Quantity|quantityRank|quantityDenseRank|maxPurchaseQuantity|
+----------+----------+--------+------------+-----------------+-------------------+
|     14096|2011-08-30|       1|           2|                2|                  2|
|     14096|2011-08-30|      -1|           7|                3|                  2|
|     14096|2011-08-30|       1|           2|                2|                  2|
|     14096|2011-08-30|       2|           1|                1|                  2|
|     14096|2011-08-30|       1|           2|                2|                  2|
|     14096|2011-08-30|       1|           2|                2|                  2|
|     14096|2011-08-30|       1|           2|                2|                  2|
|     14096|2011-09-02|       1|           5|                3|                 14|
|     14096|2011-09-02|       1|           5|                3|             

In [72]:
# COMMAND ----------

dfNoNull = dfWithDate.drop()
dfNoNull.createOrReplaceTempView("dfNoNull")

In [78]:
dfNoNull.groupBy("CustomerId","StockCode").sum("Quantity")\
.orderBy(desc("CustomerId"),desc("StockCode")).show(10)

+----------+---------+-------------+
|CustomerId|StockCode|sum(Quantity)|
+----------+---------+-------------+
|     18287|    85173|           48|
|     18287|   85040A|           48|
|     18287|   85039B|          120|
|     18287|   85039A|           96|
|     18287|    84920|            4|
|     18287|    84584|            6|
|     18287|   84507C|            6|
|     18287|   72351B|           24|
|     18287|   72351A|           24|
|     18287|   72349B|           60|
+----------+---------+-------------+
only showing top 10 rows



In [94]:
grouping_sets1 = spark.sql("""
SELECT CustomerId, StockCode, sum(Quantity) FROM dfNoNull
GROUP BY CustomerId, StockCode GROUPING SETS((CustomerId, StockCode))
ORDER BY CustomerId DESC, StockCode DESC
""")
grouping_sets2 = spark.sql("""
SELECT CustomerId, StockCode, sum(Quantity) FROM dfNoNull
GROUP BY CustomerId, StockCode
ORDER BY CustomerId DESC, StockCode DESC
""")
grouping_sets3 = spark.sql("""
SELECT CustomerId, StockCode, sum(Quantity) FROM dfNoNull
GROUP BY CustomerId, StockCode GROUPING SETS((CustomerId, StockCode),())
ORDER BY CustomerId DESC, StockCode DESC
""")

In [98]:
# GROUPING SETS((CustomerId, StockCode)) What is the use of it
# Difference of above grouping_sets1, grouping_sets2, grouping_sets3

In [96]:
dfNoNull.columns

['InvoiceNo',
 'StockCode',
 'Description',
 'Quantity',
 'InvoiceDate',
 'UnitPrice',
 'CustomerID',
 'Country',
 'date']

In [97]:
grouping_sets1.show(10)
grouping_sets2.show(10)
grouping_sets3.show(10)

+----------+---------+-------------+
|CustomerId|StockCode|sum(Quantity)|
+----------+---------+-------------+
|     18287|    85173|           48|
|     18287|   85040A|           48|
|     18287|   85039B|          120|
|     18287|   85039A|           96|
|     18287|    84920|            4|
|     18287|    84584|            6|
|     18287|   84507C|            6|
|     18287|   72351B|           24|
|     18287|   72351A|           24|
|     18287|   72349B|           60|
+----------+---------+-------------+
only showing top 10 rows

+----------+---------+-------------+
|CustomerId|StockCode|sum(Quantity)|
+----------+---------+-------------+
|     18287|    85173|           48|
|     18287|   85040A|           48|
|     18287|   85039B|          120|
|     18287|   85039A|           96|
|     18287|    84920|            4|
|     18287|    84584|            6|
|     18287|   84507C|            6|
|     18287|   72351B|           24|
|     18287|   72351A|           24|
|     18287|

In [104]:
dfNoNull.rollup("Date", "Country").agg(sum("Quantity"))\
  .selectExpr("Date", "Country", "`sum(Quantity)` as total_quantity")\
  .orderBy("Country","Date").show()

+----------+-------+--------------+
|      Date|Country|total_quantity|
+----------+-------+--------------+
|      null|   null|       5176450|
|2010-12-01|   null|         26814|
|2010-12-02|   null|         21023|
|2010-12-03|   null|         14830|
|2010-12-05|   null|         16395|
|2010-12-06|   null|         21419|
|2010-12-07|   null|         24995|
|2010-12-08|   null|         22741|
|2010-12-09|   null|         18431|
|2010-12-10|   null|         20297|
|2010-12-12|   null|         10565|
|2010-12-13|   null|         17623|
|2010-12-14|   null|         20098|
|2010-12-15|   null|         18229|
|2010-12-16|   null|         29632|
|2010-12-17|   null|         16069|
|2010-12-19|   null|          3795|
|2010-12-20|   null|         14965|
|2010-12-21|   null|         15467|
|2010-12-22|   null|          3192|
+----------+-------+--------------+
only showing top 20 rows



In [109]:
# COMMAND ----------
rolledUpDF = dfNoNull.rollup("Date", "Country").agg(sum("Quantity"))\
  .selectExpr("Date", "Country", "`sum(Quantity)` as total_quantity")\
  .orderBy("Date","Country")

rolledUpDF1 = dfNoNull.rollup("Country", "Date").agg(sum("Quantity"))\
  .selectExpr("Date", "Country", "`sum(Quantity)` as total_quantity")\
  .orderBy("Date","Country")

rolledUpDF.show()

+----------+--------------+--------------+
|      Date|       Country|total_quantity|
+----------+--------------+--------------+
|      null|          null|       5176450|
|2010-12-01|          null|         26814|
|2010-12-01|     Australia|           107|
|2010-12-01|          EIRE|           243|
|2010-12-01|        France|           449|
|2010-12-01|       Germany|           117|
|2010-12-01|   Netherlands|            97|
|2010-12-01|        Norway|          1852|
|2010-12-01|United Kingdom|         23949|
|2010-12-02|          null|         21023|
|2010-12-02|          EIRE|             4|
|2010-12-02|       Germany|           146|
|2010-12-02|United Kingdom|         20873|
|2010-12-03|          null|         14830|
|2010-12-03|       Belgium|           528|
|2010-12-03|          EIRE|          2575|
|2010-12-03|        France|           239|
|2010-12-03|       Germany|           170|
|2010-12-03|         Italy|           164|
|2010-12-03|        Poland|           140|
+----------

In [112]:
rolledUpDF1.where("Date IS NULL").orderBy("Country").show()
rolledUpDF1.where("Country IS NULL").orderBy("Country").show()

+----+------------------+--------------+
|Date|           Country|total_quantity|
+----+------------------+--------------+
|null|              null|       5176450|
|null|         Australia|         83653|
|null|           Austria|          4827|
|null|           Bahrain|           260|
|null|           Belgium|         23152|
|null|            Brazil|           356|
|null|            Canada|          2763|
|null|   Channel Islands|          9479|
|null|            Cyprus|          6317|
|null|    Czech Republic|           592|
|null|           Denmark|          8188|
|null|              EIRE|        142637|
|null|European Community|           497|
|null|           Finland|         10666|
|null|            France|        110480|
|null|           Germany|        117448|
|null|            Greece|          1556|
|null|         Hong Kong|          4769|
|null|           Iceland|          2458|
|null|            Israel|          4353|
+----+------------------+--------------+
only showing top

In [108]:
rolledUpDF.groupBy("Country").sum("total_quantity").orderBy("Country").show()

+------------------+-------------------+
|           Country|sum(total_quantity)|
+------------------+-------------------+
|              null|           10352900|
|         Australia|              83653|
|           Austria|               4827|
|           Bahrain|                260|
|           Belgium|              23152|
|            Brazil|                356|
|            Canada|               2763|
|   Channel Islands|               9479|
|            Cyprus|               6317|
|    Czech Republic|                592|
|           Denmark|               8188|
|              EIRE|             142637|
|European Community|                497|
|           Finland|              10666|
|            France|             110480|
|           Germany|             117448|
|            Greece|               1556|
|         Hong Kong|               4769|
|           Iceland|               2458|
|            Israel|               4353|
+------------------+-------------------+
only showing top

In [106]:
rolledUpDF.where("Country IS NULL").show()
rolledUpDF.where("Date IS NULL").show()
rolledUpDF.where("Country IS NULL AND Date IS NULL").show()

+----------+-------+--------------+
|      Date|Country|total_quantity|
+----------+-------+--------------+
|      null|   null|       5176450|
|2010-12-01|   null|         26814|
|2010-12-02|   null|         21023|
|2010-12-03|   null|         14830|
|2010-12-05|   null|         16395|
|2010-12-06|   null|         21419|
|2010-12-07|   null|         24995|
|2010-12-08|   null|         22741|
|2010-12-09|   null|         18431|
|2010-12-10|   null|         20297|
|2010-12-12|   null|         10565|
|2010-12-13|   null|         17623|
|2010-12-14|   null|         20098|
|2010-12-15|   null|         18229|
|2010-12-16|   null|         29632|
|2010-12-17|   null|         16069|
|2010-12-19|   null|          3795|
|2010-12-20|   null|         14965|
|2010-12-21|   null|         15467|
|2010-12-22|   null|          3192|
+----------+-------+--------------+
only showing top 20 rows

+----+-------+--------------+
|Date|Country|total_quantity|
+----+-------+--------------+
|null|   null|      

In [116]:
# COMMAND ----------

from pyspark.sql.functions import sum

cubed = dfNoNull.cube("Date", "Country").agg(sum(col("Quantity")))\
  .select("Date", "Country", "sum(Quantity)").orderBy("Date", "Country")

In [117]:
cubed.show()

+----+------------------+-------------+
|Date|           Country|sum(Quantity)|
+----+------------------+-------------+
|null|              null|      5176450|
|null|         Australia|        83653|
|null|           Austria|         4827|
|null|           Bahrain|          260|
|null|           Belgium|        23152|
|null|            Brazil|          356|
|null|            Canada|         2763|
|null|   Channel Islands|         9479|
|null|            Cyprus|         6317|
|null|    Czech Republic|          592|
|null|           Denmark|         8188|
|null|              EIRE|       142637|
|null|European Community|          497|
|null|           Finland|        10666|
|null|            France|       110480|
|null|           Germany|       117448|
|null|            Greece|         1556|
|null|         Hong Kong|         4769|
|null|           Iceland|         2458|
|null|            Israel|         4353|
+----+------------------+-------------+
only showing top 20 rows



In [118]:
cubed.where("Country IS NULL").show()
cubed.where("Date IS NULL").show()
cubed.where("Country IS NULL AND Date IS NULL").show()

+----------+-------+-------------+
|      Date|Country|sum(Quantity)|
+----------+-------+-------------+
|      null|   null|      5176450|
|2010-12-01|   null|        26814|
|2010-12-02|   null|        21023|
|2010-12-03|   null|        14830|
|2010-12-05|   null|        16395|
|2010-12-06|   null|        21419|
|2010-12-07|   null|        24995|
|2010-12-08|   null|        22741|
|2010-12-09|   null|        18431|
|2010-12-10|   null|        20297|
|2010-12-12|   null|        10565|
|2010-12-13|   null|        17623|
|2010-12-14|   null|        20098|
|2010-12-15|   null|        18229|
|2010-12-16|   null|        29632|
|2010-12-17|   null|        16069|
|2010-12-19|   null|         3795|
|2010-12-20|   null|        14965|
|2010-12-21|   null|        15467|
|2010-12-22|   null|         3192|
+----------+-------+-------------+
only showing top 20 rows

+----+------------------+-------------+
|Date|           Country|sum(Quantity)|
+----+------------------+-------------+
|null|        

In [119]:
from pyspark.sql.functions import grouping_id

In [135]:
cubed1 = dfNoNull.cube("CustomerId", "StockCode").agg(grouping_id(), sum(col("Quantity")))\
  .select("grouping_id()","CustomerId", "StockCode", "sum(Quantity)")

In [136]:
cubed1.show()

+-------------+----------+---------+-------------+
|grouping_id()|CustomerId|StockCode|sum(Quantity)|
+-------------+----------+---------+-------------+
|            0|     17809|    22961|           72|
|            0|     16098|    22726|           40|
|            0|     17548|    22553|          -24|
|            0|     14307|    22736|           10|
|            0|     17908|    21811|            1|
|            0|     14729|    21427|            2|
|            0|     12433|    22315|           24|
|            0|     14594|    21143|            3|
|            0|     16928|    22774|          138|
|            0|     17855|    22037|           12|
|            0|     18041|    22196|           30|
|            0|     18041|    21328|            2|
|            0|     16244|    22352|            6|
|            0|     14449|    22147|            8|
|            0|     16781|   82494L|            6|
|            0|     17581|    22961|           24|
|            0|     17838|   79

In [139]:
cubed1.where(col("grouping_id()") == 0).show()
cubed1.where(col("grouping_id()") == 1).show()
cubed1.where(col("grouping_id()") == 2).show()
cubed1.where(col("grouping_id()") == 3).show()

+-------------+----------+---------+-------------+
|grouping_id()|CustomerId|StockCode|sum(Quantity)|
+-------------+----------+---------+-------------+
|            0|     17340|    21172|           12|
|            0|     12428|    22309|            6|
|            0|     15747|   85099F|           33|
|            0|     13588|    22025|           12|
|            0|     14646|    22899|          500|
|            0|     12994|    21977|           24|
|            0|     13158|    21429|           16|
|            0|     14911|    23168|          132|
|            0|     14849|    22090|           12|
|            0|     16045|    23547|           25|
|            0|     17516|    23209|            4|
|            0|     17858|    23096|           26|
|            0|     17049|    48194|            4|
|            0|     15376|    22726|            2|
|            0|     15376|    23345|            6|
|            0|     14658|    21929|            3|
|            0|     15271|    2

In [145]:
dfWithDate.columns

['InvoiceNo',
 'StockCode',
 'Description',
 'Quantity',
 'InvoiceDate',
 'UnitPrice',
 'CustomerID',
 'Country',
 'date']

In [146]:
dfWithDate.show(3)

+---------+---------+--------------------+--------+--------------+---------+----------+--------------+----------+
|InvoiceNo|StockCode|         Description|Quantity|   InvoiceDate|UnitPrice|CustomerID|       Country|      date|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+----------+
|   536365|   85123A|WHITE HANGING HEA...|       6|12/1/2010 8:26|     2.55|     17850|United Kingdom|2010-12-01|
|   536365|    71053| WHITE METAL LANTERN|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|2010-12-01|
|   536365|   84406B|CREAM CUPID HEART...|       8|12/1/2010 8:26|     2.75|     17850|United Kingdom|2010-12-01|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+----------+
only showing top 3 rows



In [147]:
# COMMAND ----------
pivoted = dfWithDate.groupBy("Date").pivot("Country").sum()
# COMMAND ----------

In [148]:
pivoted.columns

['Date',
 'Australia_sum(CAST(Quantity AS BIGINT))',
 'Australia_sum(UnitPrice)',
 'Australia_sum(CAST(CustomerID AS BIGINT))',
 'Austria_sum(CAST(Quantity AS BIGINT))',
 'Austria_sum(UnitPrice)',
 'Austria_sum(CAST(CustomerID AS BIGINT))',
 'Bahrain_sum(CAST(Quantity AS BIGINT))',
 'Bahrain_sum(UnitPrice)',
 'Bahrain_sum(CAST(CustomerID AS BIGINT))',
 'Belgium_sum(CAST(Quantity AS BIGINT))',
 'Belgium_sum(UnitPrice)',
 'Belgium_sum(CAST(CustomerID AS BIGINT))',
 'Brazil_sum(CAST(Quantity AS BIGINT))',
 'Brazil_sum(UnitPrice)',
 'Brazil_sum(CAST(CustomerID AS BIGINT))',
 'Canada_sum(CAST(Quantity AS BIGINT))',
 'Canada_sum(UnitPrice)',
 'Canada_sum(CAST(CustomerID AS BIGINT))',
 'Channel Islands_sum(CAST(Quantity AS BIGINT))',
 'Channel Islands_sum(UnitPrice)',
 'Channel Islands_sum(CAST(CustomerID AS BIGINT))',
 'Cyprus_sum(CAST(Quantity AS BIGINT))',
 'Cyprus_sum(UnitPrice)',
 'Cyprus_sum(CAST(CustomerID AS BIGINT))',
 'Czech Republic_sum(CAST(Quantity AS BIGINT))',
 'Czech Republic_

In [156]:
pivoted.where("Date > '2011-12-05'").select("Date", 'USA_sum(CAST(Quantity AS BIGINT))').show()

+----------+---------------------------------+
|      Date|USA_sum(CAST(Quantity AS BIGINT))|
+----------+---------------------------------+
|2011-12-06|                             null|
|2011-12-09|                             null|
|2011-12-08|                             -196|
|2011-12-07|                             null|
+----------+---------------------------------+

